In [1]:
import json
import os
import random

In [2]:
import pandas as pd

In [3]:
STATEMENTS_DIR = "/Users/YK/mt/project/statements_2/"
RACE_PART = "train/middle"
RACE_DIR = "/Users/YK/mt/RACE"
PARSED_RACE_DIR = "/Users/YK/mt/parsed/race"
LINK_TEMPLATE = "<html><a href={link}>text</a></html>"

In [4]:
rule_names = [
    file_name for file_name in os.listdir(STATEMENTS_DIR) 
        if file_name[0] != '.' and os.path.isdir(os.path.join(STATEMENTS_DIR, file_name))
]

In [5]:
all_statements = []

for rule_name in rule_names:
    directory = os.path.join(STATEMENTS_DIR, rule_name, RACE_PART)
    for file_name in os.listdir(directory):
        with open(os.path.join(directory, file_name), "rt") as f:
            file_no_str = file_name.split(".", 1)[0]
#             text_file = LINK_TEMPLATE.format(
#                 link=f'file://{os.path.join(RACE_DIR, RACE_PART, file_no_str + ".txt")}',
#                 text=file_no_str + ".txt"
#             )
#             tree_file = LINK_TEMPLATE.format(
#                 link=f'file://{os.path.join(PARSED_RACE_DIR, RACE_PART, file_no_str + ".txt.tree")}',
#                 text=file_no_str + ".txt.tree"
#             )
            statements = json.load(f)
            for statement in statements:
                statement.update(
#                     {"text_file": text_file, "tree_file": tree_file}
                    {"text_number": file_no_str}
                )
            all_statements.extend(statements)

In [6]:
df = pd.DataFrame(all_statements)
df["reason_number"] = df.reason.apply(lambda r: r[0])
df.reason = df.reason.apply(lambda r: r[1])

In [7]:
df[
    [
        "rule", 
        "reason", 
        "statement_text", 
        "nucleus", 
        "satellite_nucleus", 
        "text_number",
        "nucleus_proximity",
        "left_boundary",
        "right_boundary",
        "reason_number"
    ]
].sort_values(
    by=["rule", "reason_number"]
).drop(
    "reason_number", 1
).to_excel(
    os.path.join(
        STATEMENTS_DIR, f"{RACE_PART.replace('/', '-')}_{random.randint(0, 2**32):x}.xlsx"
    ),
    index=False
)